In [152]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [153]:
#getting and organizing tag list
baseurl="https://api.mangadex.org/"
taglist=requests.get(baseurl+"manga/tag")
taglist=taglist.json()["data"]

taglist[0]
data=[]

for entry in taglist:
    entrydict={}
    entrydict['id']=entry['id']
    entrydict['type']=entry['type']
    entrydict['name']=entry['attributes']['name']['en']
    entrydict['group']=entry['attributes']['group']
    data.append(entrydict)

avail_tags=pd.DataFrame(data)
avail_tags.to_csv("./resources/tag_list.csv")
avail_tags[avail_tags.id=="0234a31e-a729-4e28-9d6a-3f87c4966b9e"].name[0]

'Oneshot'

In [154]:
avail_tags

,id,type,name,group
0,0234a31e-a729-4e28-9d6a-3f87c4966b9e,tag,Oneshot,format
1,07251805-a27e-4d59-b488-f0bfbec15168,tag,Thriller,genre
2,0a39b5a1-b235-4886-a747-1d05d216532d,tag,Award Winning,format
3,0bc90acb-ccc1-44ca-a34a-b9f3a73259d0,tag,Reincarnation,theme
4,256c8bd9-4904-4360-bf4f-508a76d67183,tag,Sci-Fi,genre
...,...,...,...,...
71,f4122d1c-3b44-44d0-9936-ff7502c39ad3,tag,Adaptation,format
72,f42fbf9e-188a-447b-9fdc-f19dc1e4d685,tag,Music,theme
73,f5ba408b-0e7a-484d-8d49-4e9125ac96de,tag,Full Color,format
74,f8f62932-27da-4fe4-8ee1-6779a8c5edba,tag,Tragedy,genre


In [155]:


baseurl="https://api.mangadex.org/manga"
loopnum=0
request_entries=100
manga_list=[]
publication_demographic=['shounen','shoujo','josei','seinen']
while len(manga_list)<10000:
    
    payload={'limit':100,'hasAvailableChapters':"1","offset":len(manga_list),'contentRating[]':['safe','suggestive'],'publicationDemographic[]':publication_demographic,'availableTranslatedLanguage[]':['en','es','es-la']}#,'order[]':{'updatedAt':'asc'
    data_req=requests.get(baseurl,params=payload)
    
     #url=f"https://api.mangadex.org/manga?limit=100&offset={len(manga_list)}&contentRating=safe,suggestive&&order=updatedAt=asc"

    #data_req=requests.get(url)
   
    #print(data_req.url)
    manga_data=data_req.json()["data"]
    
    for manga in manga_data:
        entrydict={}
        entrydict['id']=manga['id']
        entrydict['type']=manga['type']
        title_lang=list(manga['attributes']['title'].keys())[0]
        entrydict['title']=manga['attributes']['title'][title_lang]
        entrydict['orig_lang']=manga['attributes']['originalLanguage']
        entrydict['target_demographic']=manga['attributes']['publicationDemographic']
        #entrydict['total_chapters']=manga['attributes']['lastChapter']
        tags=[]
        for tag in manga['attributes']['tags']:
            tagname=avail_tags[avail_tags.id==tag['id']].name
            #tagname=tagname.str.replace("'","").replace(" ","-")
            tagname=tagname.tolist()[0]
            
            tags.append(tagname)

        entrydict['tags']=tags
        manga_list.append(entrydict)
    request_entries=len(manga_data)
    loopnum=loopnum+1
    sleep(.02)
#



In [156]:
manga_df=pd.DataFrame(manga_list)
for element in avail_tags.name:
    
    manga_df[element]=manga_df["tags"].apply(lambda x:element in x)



manga_df.to_pickle('./resources/manga_list.pckl')
manga_df.head()

,id,type,title,orig_lang,target_demographic,tags,Oneshot,Thriller,Award Winning,Reincarnation,...,Slice of Life,Aliens,Cooking,Supernatural,Mystery,Adaptation,Music,Full Color,Tragedy,Gyaru
0,5fb002de-95d1-4acb-a1ba-aabcfec814c3,manga,Hell's Boat 136,ja,seinen,"[Action, Martial Arts, Gore]",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,22290888-46b5-4f42-992c-5c2934fe91d1,manga,"I’m a Human, but More So a Vampire",zh,shounen,"[Long Strip, Adventure, Horror, Fantasy, Vampi...",False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,ab41d97a-0725-4f37-85db-da001d4ff0dc,manga,Chi no Wadachi,ja,seinen,"[Psychological, Drama, School Life, Horror, Sl...",False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,2c3cd370-04bd-4994-87dc-e54a4d80c4ec,manga,Natsume Yuujinchou,ja,shoujo,"[Demons, Comedy, Drama, Slice of Life, Superna...",False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,8325137c-32bf-464c-b716-b4c9b17c0002,manga,Natsume Arata no Kekkon,ja,seinen,"[Psychological, Romance, Comedy, Drama, Mystery]",False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [157]:
manga_df[manga_df['Reincarnation']].head()

,id,type,title,orig_lang,target_demographic,tags,Oneshot,Thriller,Award Winning,Reincarnation,...,Slice of Life,Aliens,Cooking,Supernatural,Mystery,Adaptation,Music,Full Color,Tragedy,Gyaru
19,6e769716-df1e-4d66-89cc-32f3b46e17dc,manga,Damn Reincarnation,ko,shounen,"[Reincarnation, Monsters, Action, Demons, Long...",False,False,False,True,...,False,False,False,False,False,True,False,True,False,False
21,cbb22826-963e-4602-a5cd-a50c75723d59,manga,Maseki Gurume: Mamono no Chikara wo Tabeta Ore...,ja,shounen,"[Reincarnation, Action, Romance, Adventure, Ma...",False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
22,c9bed72c-d680-4409-a954-5fdadde27b74,manga,"Shi ni Modori, Subete wo Sukuu Tame ni Saikyou...",ja,shounen,"[Reincarnation, Time Travel, Monsters, Action,...",False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
25,56ac4bec-0e20-4fd3-9c12-74eb9cfc9adc,manga,Ochikobore kuni o deru ~ jitsuwa sekai de 4 hi...,ja,shounen,"[Reincarnation, Monsters, Action, Demons, Come...",False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
27,c904ac3b-6437-46f8-94cf-b21f72bc3f54,manga,"Kouritsu-Chuu Madoushi, Daini no Jinsei de Mad...",ja,seinen,"[Reincarnation, Time Travel, Action, Romance, ...",False,False,False,True,...,False,False,False,False,False,True,False,False,False,False


In [163]:
manga_df.groupby('orig_lang').count()


,id,type,title,target_demographic,tags,Oneshot,Thriller,Award Winning,Reincarnation,Sci-Fi,...,Supernatural,Mystery,Adaptation,Music,Full Color,Tragedy,Gyaru,avg_rating,bayesian_rating,follows
orig_lang,,,,,,,,,,,,,,,,,,,,,
en,122,122,122,122,122,122,122,122,122,122,...,122,122,122,122,122,122,122,121,122,122
es-la,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
fr,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
id,29,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,25,29,29
it,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ja,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459,...,7459,7459,7459,7459,7459,7459,7459,7359,7459,7459
ko,1250,1250,1250,1250,1250,1250,1250,1250,1250,1250,...,1250,1250,1250,1250,1250,1250,1250,1231,1250,1250
lt,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
ms,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [158]:
manga_df=pd.read_pickle('./resources/manga_list.pckl')
avg_rating=[]
bay_rating=[]
follows=[]
startloc=0
endloc=100

baseurl="https://api.mangadex.org/statistics/manga"
while endloc<=10001:

    
    
    manga_id_list=manga_df['id'][startloc:endloc]
    payload={'manga[]':manga_id_list}
    data_req=requests.get(baseurl,params=payload).json()

    try:
        for element in manga_id_list:
            avg_rating_elem=data_req['statistics'][element]['rating']['average']
            bay_rating_elem=data_req['statistics'][element]['rating']['bayesian']
            follows_elem=data_req['statistics'][element]['follows']
            avg_rating.append(avg_rating_elem)
            bay_rating.append(bay_rating_elem)
            follows.append(follows_elem)
    except:
        print(f"Error {data_req['errors'][0]['status']}: {data_req['errors'][0]['detail']}")
    
    startloc=startloc+100
    endloc=endloc+100


manga_df['avg_rating']=avg_rating
manga_df['bayesian_rating']=bay_rating
manga_df['follows']=follows
manga_df.to_pickle('./resources/manga_list.pckl')

In [159]:

baseurl="https://api.mangadex.org/statistics/manga"
manga_id_list=['91e957f7-7865-426f-801d-b57625622c72','ccb654fd-d4de-44d1-a9c1-f58ba34f3510']
payload={'manga[]':manga_id_list}
data_req=requests.get(baseurl,params=payload)
data_req.json()['statistics']['91e957f7-7865-426f-801d-b57625622c72']

{'rating': {'average': 8.2229, 'bayesian': 8.15446053410934}, 'follows': 3789}

In [160]:
manga_df


,id,type,title,orig_lang,target_demographic,tags,Oneshot,Thriller,Award Winning,Reincarnation,...,Supernatural,Mystery,Adaptation,Music,Full Color,Tragedy,Gyaru,avg_rating,bayesian_rating,follows
0,5fb002de-95d1-4acb-a1ba-aabcfec814c3,manga,Hell's Boat 136,ja,seinen,"[Action, Martial Arts, Gore]",False,False,False,False,...,False,False,False,False,False,False,False,10.0000,8.059481,89
1,22290888-46b5-4f42-992c-5c2934fe91d1,manga,"I’m a Human, but More So a Vampire",zh,shounen,"[Long Strip, Adventure, Horror, Fantasy, Vampi...",False,False,False,False,...,False,False,False,False,False,False,False,5.2500,7.832589,257
2,ab41d97a-0725-4f37-85db-da001d4ff0dc,manga,Chi no Wadachi,ja,seinen,"[Psychological, Drama, School Life, Horror, Sl...",False,False,False,False,...,False,False,False,False,False,False,False,8.6987,8.657656,21525
3,2c3cd370-04bd-4994-87dc-e54a4d80c4ec,manga,Natsume Yuujinchou,ja,shoujo,"[Demons, Comedy, Drama, Slice of Life, Superna...",False,False,False,False,...,True,False,False,False,False,False,False,9.3191,9.139842,6262
4,8325137c-32bf-464c-b716-b4c9b17c0002,manga,Natsume Arata no Kekkon,ja,seinen,"[Psychological, Romance, Comedy, Drama, Mystery]",False,False,False,False,...,False,True,False,False,False,False,False,8.5972,8.418501,4696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,b4320039-9b91-44a7-a60d-c7ba8c0684e7,manga,Beelzebub-jou no Oki ni Mesu mama.,ja,shounen,"[Romance, Comedy, Fantasy, Slice of Life]",False,False,False,False,...,False,False,False,False,False,False,False,9.1228,8.434526,2480
9996,5966861c-9e2c-41b8-a768-eba93b13b94f,manga,Ore wo Suki Nano wa Omae Dake ka yo,ja,shounen,"[Romance, Comedy, Harem, Drama, School Life]",False,False,False,False,...,False,False,False,False,False,False,False,6.7502,6.811411,20475
9997,aefff461-ce0f-4db2-b160-3eeb1b03f48e,manga,Merry Christmas,ja,seinen,"[Oneshot, Romance, Web Comic, Slice of Life, F...",True,False,False,False,...,False,False,False,False,True,False,False,7.6939,7.925728,188
9998,ccd10c9b-1aad-437f-a888-d76d64525e67,manga,"""Aishiteru"", Uso dakedo.",ja,shoujo,"[Romance, School Life]",False,False,False,False,...,False,False,False,False,False,False,False,4.2222,7.722916,192


In [161]:
tagname



'Fantasy'

In [164]:
#manga_df.loc[manga_df['Mystery']==True]
True in list(manga_df['Reincarnation'])

True